In [1]:
%%time
%%sh

# The name of our algorithm
# Makoto.Sano@Mack-the-Psych.com
algorithm_name=sagemaker-vdok3-bert

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  48.64kB
Step 1/32 : FROM nvidia/cuda:10.0-cudnn7-devel-ubuntu16.04
 ---> a2b601423810
Step 2/32 : RUN apt-get -y update && apt-get -y upgrade && apt-get install -y --no-install-recommends          curl          git          unzip          bzip2          libgl1-mesa-glx          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 97673f29f71e
Step 3/32 : RUN git clone https://github.com/pyenv/pyenv.git .pyenv
 ---> Using cache
 ---> 9ffd36aa844f
Step 4/32 : WORKDIR /
 ---> Using cache
 ---> d24a186d0b57
Step 5/32 : ENV HOME  /
 ---> Using cache
 ---> 033eb8a937e5
Step 6/32 : ENV PYENV_ROOT /.pyenv
 ---> Using cache
 ---> e557600eeede
Step 7/32 : ENV PATH $PYENV_ROOT/shims:$PYENV_ROOT/bin:$PATH
 ---> Using cache
 ---> 490d2419b55b
Step 8/32 : RUN pyenv install anaconda3-5.0.0
 ---> Using cache
 ---> 84e7f8038281
Step 9/32 : RUN pyenv global anaconda3-5.0.0
 ---> Using cache
 ---> b

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 9.49 ms, sys: 523 µs, total: 10 ms
Wall time: 4.58 s


In [2]:
# S3 prefix
# Makoto.Sano@Mack-the-Psych.com
prefix = 'vdok3_bert_trial'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Makoto.Sano@Mack-the-Psych.com
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-bert:latest'.format(account, region)
vdok3bert = sage.estimator.Estimator(image,
                       role, 1, 'ml.p3.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
vdok3bert.fit(data_location)

2020-12-28 06:54:19 Starting - Starting the training job...
2020-12-28 06:54:43 Starting - Launching requested ML instancesProfilerReport-1609138459: InProgress
......
2020-12-28 06:55:43 Starting - Preparing the instances for training.........
2020-12-28 06:57:05 Downloading - Downloading input data
2020-12-28 06:57:05 Training - Downloading the training image.....................
2020-12-28 07:00:47 Training - Training image download completed. Training in progress./.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Starting the training.
bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_typ

In [6]:
from sagemaker.predictor import csv_serializer
predictor = vdok3bert.deploy(1, 'ml.p3.2xlarge', serializer=csv_serializer)

-------------------!

In [7]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Score_Class,Definition-Score
1,0
1,1
2,1
2,2



In [8]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Score_Class,Definition-Score
1,0
1,1
2,1
2,2



In [9]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [10]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = vdok3bert.transformer(instance_count=1,
                               instance_type='ml.p3.2xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

In [11]:
transformer.transform(data_location + '/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv', content_type='text/csv', split_type='Line')
transformer.wait()

s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out".format(transform_output_folder), '/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out')
with open('/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

...........................................Starting the inference server with 8 workers.
2020/12/28 07:26:20 [crit] 42#42: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [28/Dec/2020:07:26:20 +0000] "GET /ping HTTP/1.1" 502 182 "-" "Go-http-client/1.1"
2020/12/28 07:26:20 [crit] 42#42: *3 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [28/Dec/2020:07:26:20 +0000] "GET /ping HTTP/1.1" 502 182 "-" "Go-http-client/1.1"
2020/12/28 07:26:20 [crit] 42#42: *5 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to